# X3DH Block Mappings

Generate course mappings between LHS and RHS texts of the X3DH protocol, using embeddings and vector databases.

In [1]:
import { readFileSync } from 'node:fs';

import { SystemMessage, HumanMessage } from "@langchain/core/messages";
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { OpenAIEmbeddings } from "@langchain/openai";
import { MemoryVectorStore } from "langchain/vectorstores/memory";

import { EXPERIMENTS_DIR, SERVER_DATA_DIR } from '../server/src/util/fileUtils.ts';
import { newModel } from '../server/src/agents/agent.ts';

// takes a string text and splits it into segments
// using at least two newlines as the delimiter. 
// returns an array of {segment, startIndex, endIndex}  
// with the start index and the end index of each segment
function splitText(text: string) {
  const segments = text.split('\n\n');
  const segind = [];
  let lastIndex = 0;
  for (const segment of segments) {
    segind.push({segment: segment, start: lastIndex, stop: lastIndex + segment.length});
    lastIndex += segment.length + 2; // +2 for the two newlines
  }
  const segindices = [];
  for (const {segment, start, stop} of segind) {
    if (segment !== '') {
      if (segment[0] === '\n') {
        segindices.push({segment: segment.slice(1), start: start+1, stop: stop});
      } else {
        segindices.push({segment, start, stop});
      }
    }
  }
  return segindices;
}

// summarize a given text segment into a short description
const prompt = ChatPromptTemplate.fromMessages([ new MessagesPlaceholder("messages") ]);
const llm = newModel("Anthropic");
const chain = prompt.pipe(llm);
const PROMPT = "Summarize the following text into a short description in less than 10 words. The description should be a single sentence that captures the main idea of the text. The text may contain multiple paragraphs, but the description should be concise and to the point. The description should not include any specific details or examples from the text. The description should be in English and should not include any special characters or formatting.";

const lhsText = readFileSync(`${SERVER_DATA_DIR}/X3DH/selected-text.txt`, 'utf-8');
const rhsText = readFileSync(`${SERVER_DATA_DIR}/X3DH/pre-written.txt`, 'utf-8');

const lhsSegIndices = splitText(lhsText);
const lhsSegments = [];
for (const {segment, start, stop} of lhsSegIndices) { lhsSegments.push({pageContent: segment, metadata: {start: start, stop: stop}}); }
console.log("Number of LHS segments", lhsSegments.length);

const embeddings = new OpenAIEmbeddings({ model: "text-embedding-3-large" });
const vectorStore = new MemoryVectorStore(embeddings);
await vectorStore.addDocuments(lhsSegments);

const rhsSegIndices = splitText(rhsText);
console.log("Number of RHS segments", rhsSegIndices.length);

const numResults = 2;
const mappings = [];
for (const {segment, start, stop} of rhsSegIndices) { 
  console.log("Segment:", segment);
  const output = await chain.invoke({ messages: [
    new SystemMessage(PROMPT),
    new HumanMessage(segment)
  ]});
  const description = output.content;
  const rhsRanges = [ {start: start, end: stop} ];

  const searchResults = await vectorStore.similaritySearch(segment, numResults);
  const lhsRanges = [];
  for (const doc of searchResults) {
    lhsRanges.push({start: doc.metadata.start, end: doc.metadata.stop});
  }
  const annotation = { description: description, lhsRanges: lhsRanges, rhsRanges: rhsRanges };
  console.log("Annotation:", JSON.stringify(annotation, null, 2));
  mappings.push(annotation);
}

Number of LHS segments 119
Number of RHS segments 47
Segment: use curve25519_dalek::{edwards::EdwardsPoint, scalar::Scalar};
use rand_core::{OsRng, RngCore}; 
use sha2::{Digest, Sha512};
// use hmac::{Hmac, Mac};
use hkdf::Hkdf;
Annotation: {
  "description": "Cryptographic operations using curve25519_dalek, random generation, and hashing.",
  "lhsRanges": [
    {
      "start": 2455,
      "end": 3594
    },
    {
      "start": 1778,
      "end": 1971
    }
  ],
  "rhsRanges": [
    {
      "start": 0,
      "end": 166
    }
  ]
}
Segment: // X3DH parameters
const INFO: &[u8] = b"MyProtocolImplementation";
Annotation: {
  "description": "X3DH protocol implementation parameters with INFO constant.",
  "lhsRanges": [
    {
      "start": 1690,
      "end": 1714
    },
    {
      "start": 5806,
      "end": 5829
    }
  ],
  "rhsRanges": [
    {
      "start": 168,
      "end": 235
    }
  ]
}
Segment: // Key types for the protocol
pub struct IdentityKey {
    private_key: Scalar,
    

47

In [2]:
const annotations = {mappings: mappings, lhsLabels: [], rhsLabels: []};
console.log(JSON.stringify(annotations, null, 2));


{
  "mappings": [
    {
      "description": "Cryptographic operations using curve25519_dalek, random generation, and hashing.",
      "lhsRanges": [
        {
          "start": 2455,
          "end": 3594
        },
        {
          "start": 1778,
          "end": 1971
        }
      ],
      "rhsRanges": [
        {
          "start": 0,
          "end": 166
        }
      ]
    },
    {
      "description": "X3DH protocol implementation parameters with INFO constant.",
      "lhsRanges": [
        {
          "start": 1690,
          "end": 1714
        },
        {
          "start": 5806,
          "end": 5829
        }
      ],
      "rhsRanges": [
        {
          "start": 168,
          "end": 235
        }
      ]
    },
    {
      "description": "Cryptographic identity key with private and public components.",
      "lhsRanges": [
        {
          "start": 4542,
          "end": 4796
        },
        {
          "start": 14219,
          "end": 14440
        }


In [ ]:
import { readFileSync } from 'node:fs';
import { EXPERIMENTS_DIR, SERVER_DATA_DIR } from '../server/src/util/fileUtils.ts';

//const lhsText = readFileSync(`${EXPERIMENTS_DIR}/20250409-latex-to-markdown/selected-AES.md`, 'utf-8');
const lhsText = readFileSync(`${SERVER_DATA_DIR}/AES-md/selected-text.txt`, 'utf-8');
const rhsText = readFileSync(`${SERVER_DATA_DIR}/AES-md/pre-written.txt`, 'utf-8');
console.log("##"+lhsText.substring(31,161)+"##");

##The general function for executing AES-128, AES-192, or AES-256 is denoted by CIPHER(); its inverse is denoted by INVCIPHER().[^1]##
